In [1]:
#%config IPCompleter.greedy=True

# Logistic Regression Model

### Base code: Julien Courbebaisse
### Revisions and modifications: Matheus Faria and Luiz Resende Silva 

The entire code was implemented using two main libraries to handle data and perform the calculations: Pandas and NumPy.

In [2]:
import numpy as np
import pandas as pd
import time
import timeit

Firstly, the function was implemented to be used to import the data sets, remove malformed values and extract basic statistics and pairwise correlation coerfficients between the columns (features and true labels)

In [3]:
def Read_File_DF(File_Name, separation = ",", head = None, replace = [], drop=True):
    try:
        separation = separation.lower()
        if(separation == "space" or separation == "tab"):
            separation = "\t"
        Raw_Data_Set = pd.read_csv(File_Name, delimiter=separation, header=head, na_values=replace)
        RawRowsColumns = Raw_Data_Set.shape
        if(replace != None):
            Missing = Raw_Data_Set.isnull().sum().sum()
            print("Total number of missing/anomalous 'entries' in the data set: ",Missing)
            if(drop == True):
                Raw_Data_Set.dropna(axis=0, how='any', inplace=True)
                CleanRowsColumns = Raw_Data_Set.shape
                print("Number of examples with missing values deleted from data set: ",(RawRowsColumns[0]-CleanRowsColumns[0]))
        Data_Set = Raw_Data_Set.to_numpy()
        return Data_Set
    except:
        print("READ_FILE_ERROR\n")

Other few functions were implemented to perform the binary classifiacation of the true labels from the data sets. The function were designed to be more general and allow modifications on both datasets

In [4]:
#Function replaces both by boundary (False) and fixed values (True) - each will be used in a different data set
def replace_class(vector, index, flag=False):
    if(flag == True):
        if vector[index]==4:
            vector[index]=1
            return vector
        elif(vector[index]==2):
            vector[index]=0
            return vector
    elif(flag == False):
        if (vector[index]<=5):
            vector[index]=0
            return vector
        elif(vector[index]>5):
            vector[index]=1
            return vector

#Function either adds (False) or substitutes (True) column by the intercept bias weights
def substi_add(dataset, index, flag=True): #Adding column of 1 to determine bias term
    if(flag==True):
        for element in dataset:
            element[index] = 1
        return dataset
    else:
        temp = dataset
        temp = np.insert(temp, 0, 1, axis=1)
        return temp    

In [5]:
def preproc(Dataset, Index1, Index2=None, Flag=True):
    if(Flag==True):
        for element in Dataset:
            element = replace_class(element,index=Index1, flag=Flag)
        Dataset = substi_add(Dataset, index=Index2, flag=Flag)
        return Dataset
    else:
        for element in Dataset:
            element = replace_class(element,index=Index1, flag=Flag)
        Dataset = substi_add(Dataset, index=Index2, flag=Flag)
        return Dataset

Another function was implemented to retrieve information about the data sets, such as general statistics and Spearman Rank Correlation between columns

In [6]:
def Data_Stats(Data, QQ_DD = True, show = True):
    try:
        Data_Set = pd.DataFrame(Data, index = (list(range(0,Data.shape[0]))), columns = (list(range(0,Data.shape[1]))))
        if(QQ_DD == True):           
            quantiles = [0.00, 0.25, 0.50, 0.75] #Calculating quartiles
        else:
            quantiles = [0.00, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 1.00] #Calculating quartiles

        Describ = Data_Set.describe(percentiles = quantiles) #Data set general stats description
        Correlation = Data_Set.corr('spearman') #Computing pairwise feature correlation through Spearman rank correlation
        name = ("GeneralStats.xlsx")
        with pd.ExcelWriter(name) as writer: #Outputting Excel file with statistics
            Describ.to_excel(writer, sheet_name='Data_Description')
            Correlation.to_excel(writer, sheet_name='Column_Correlation')
        if(show == True):
            print(Data_Set)
            print(Describ) #Printing statistics to screen
            print(Correlation) #Printing statistics to screen
    except:
        print("STATS_FUNCTION_ERROR\n")

Lastly, the Linear Classification model was implemented as a Python class. The class was called logistiReg and all the functions needed to perform the Logistic Regression were implemented inside it

In [7]:
class logisticReg(object):
    """Class to build logistic regression model"""
        
    #Constructor called passing data set
    def __init__(self, dataset):
        self.dataset = dataset
    
    #Sigmoid function
    def sigmoid(self, a):
        sigma = (1/(1+(np.exp(-a))))
        return sigma
        
    #Update rule function
    def update(self, weights, dataset, Lrate):
        new_weights=weights #Initializing weights vector
        suma=0
        for element in dataset:
            suma += (((element[(self.Features_Selec)])*((element[(self.Outcome_Col)])-(self.sigmoid(new_weights.dot(element[(self.Features_Selec)]))))))
        new_weights = np.add(new_weights, (Lrate*suma)) #Updating weights
        return new_weights

    #Function to calculate the Cross-Entropy Loss
    def Cross_Entropy(self , data, weight):
        CE = 0
        for element in data:
            Term1 = np.dot(element[(self.Outcome_Col)], np.log(self.sigmoid(weight.dot(element[(self.Features_Selec)]))))
            Term2 = np.dot((1 - element[(self.Outcome_Col)]),np.log(1 - self.sigmoid(weight.dot(element[(self.Features_Selec)]))))
            CE += - np.add(Term1,Term2)
        return CE
    
    
    #Fitting model function    
    def fit_model(self, Training_Set, lstFeatures, outCol, rate=0.00001, maxIter=1000, reduct=1, ran=False):
        self.Features_Selec = lstFeatures #List of features selected to fit the model
        self.Outcome_Col = outCol #Index of outcome column
        
        if(ran == True):
            w = np.random.randint(0,10,len(self.Features_Selec)) #Starting vector of weights with random numbers between 0-10
        else:
            w = np.zeros(len(self.Features_Selec)) #Starting vector of weights with zeros
        CE_D = []
        i=0
        while i < maxIter:
            w = self.update(weights=w, dataset=Training_Set, Lrate=rate)
            cost = self.Cross_Entropy(data=Training_Set, weight=w)
            CE_D.append(cost)
            rate = rate/reduct
            i+=1
#        print(CE_D)
        return w
    
    #Function to use the weights calculated to perform predictions
    def predict(self, data, weights):
        truelabel = []
        predicted = []
        compare = []
        for example in data:
            realpred = np.dot(example[self.Features_Selec],weights)
            if(realpred <= 0.5):
                predicted.append(int(0))
            else:
                predicted.append(int(1))
            truelabel.append(int(example[self.Outcome_Col]))
        
        compare.append(truelabel)
        compare.append(predicted)
        return compare    

    def evaluate_acc(self, Preds):
        score = 0
        TP = 0
        FN = 0
        Predictions = np.transpose(Preds)
        for pred in Predictions:
            if pred[1]==pred[0]:
                score+=1
                if(pred[1]==1):
                    TP += 1
            else:
                if(pred[1]==0):
                    FN += 1
        
        print("Correct Predictions: ",score)
        print("Total Predictions: ",len(Predictions))
        print("Accuracy Percentage: ",round(((score/len(Predictions))*100),4),"%")
        accur = ((score/len(Predictions))*100)
        if(TP>0 or FN>0):
            print("Sensitivity Percentage: ",round(((TP/(TP+FN))*100),4),"%")
            sense = ((TP/(TP+FN))*100)
        else:
            print("Sensitivity Percentage: 0.0000%")
            sense = 0
        return accur, sense
    
    def kfold(self,k, data, LstFeatures, OutCol, Rate=0.00001, MaxIter=1000, Reduct=1, Ran=False):
        i=0
        score1=0
        score2=0
        batch_size=np.floor(data[:,1].size/k)
        while i < k: 
            if(i<k-1):
                training_data=np.delete(data, np.s_[(int)(i*batch_size):(int)((i+1)*batch_size)], 0)
                test_data= data[(int)(i*(batch_size)):(int)((i+1)*(batch_size))]
                weights = self.fit_model(training_data, LstFeatures, OutCol, rate=Rate, maxIter=MaxIter, reduct=Reduct, ran=Ran)
                preds = self.predict(test_data, weights)
                print("\nSegmentation #",i+1," Results:")
                temp1, temp2 = self.evaluate_acc(preds)
                score1+=temp1
                score2+=temp2
                i+=1
            else:
                training_data=np.delete(data, np.s_[(int)(i*batch_size):], 0)
                test_data= data[(int)(i*(batch_size)):]
                weights = self.fit_model(training_data, LstFeatures, OutCol, rate=Rate, maxIter=MaxIter, reduct=Reduct, ran=Ran)
                preds = self.predict(test_data, weights)
                print("\nSegmentation #",i+1," Results:")
                temp1, temp2 = self.evaluate_acc(preds)
                score1+=temp1
                score2+=temp2
                i+=1
        print("\n##### RESULTS FOR THE kFOLD CROSS VALIDATION WERE #####")
        print("Average Accuracy:",round((score1/k),3),"%")
        print("Average Sensitivity:",round((score2/k),3),"%")

With the above class and functions, we can perform the Logistic Regression in each of the data sets

## Breast Cancer Data Set

In the breast cancer data set, only 16 instances were found with incorrect values ("?"). These examples were removed from the data set when importing by flagging these "?" characters

In [8]:
#Load data and remove missing values
filename = 'breast-cancer-wisconsin.data'
databc = Read_File_DF(filename, separation=',', head=None, replace=["?"], drop=True)
databc

Total number of missing/anomalous 'entries' in the data set:  0
Number of examples with missing values deleted from data set:  0


array([[1000025,       5,       1, ...,       1,       1,       2],
       [1002945,       5,       4, ...,       2,       1,       2],
       [1015425,       3,       1, ...,       1,       1,       2],
       ...,
       [ 888820,       5,      10, ...,      10,       2,       4],
       [ 897471,       4,       8, ...,       6,       1,       4],
       [ 897471,       4,       8, ...,       4,       1,       4]],
      dtype=int64)

In the breast cancer data, the first column referent to the sample IDs was replaced by the intercept values since these information was not needed. Therefore, for this data set, the first weight in the vector $ w_{k}$ containing $k=m+1$ weights will be the intercept. The resulting data matrix will have6 683 rows and 11 columns: 9 features plus 1 intercept and the true labels

In [9]:
databc = preproc(databc, Index1=10, Index2=0, Flag=True)
databc

array([[ 1,  5,  1, ...,  1,  1,  0],
       [ 1,  5,  4, ...,  2,  1,  0],
       [ 1,  3,  1, ...,  1,  1,  0],
       ...,
       [ 1,  5, 10, ..., 10,  2,  1],
       [ 1,  4,  8, ...,  6,  1,  1],
       [ 1,  4,  8, ...,  4,  1,  1]], dtype=int64)

Statistics on the data set

In [10]:
Data_Stats(databc, QQ_DD = True, show = True)

STATS_FUNCTION_ERROR



Calling constructor for the 

In [11]:
modelBC = logisticReg(databc)

Defining the set/subset of features to be selected (_always between 0-9 with the 0 included_ - $w_{0}$)

In [12]:
feats = [0,1,2,3,4,5,6,7,8,9,10]

Defining the set/subset of examples to be used in training and validating the model ( _for uses outside kFold method_ )

In [13]:
training = databc[:400,:]
testing = databc[500:750,:]

Running model outside kFold method ( _single training and validation_ )

In [14]:
start = timeit.default_timer() #Starting clock to count time for training

weights = modelBC.fit_model(Training_Set=training, lstFeatures=feats, outCol=10, rate=0.00001, maxIter=1000, reduct=1, ran=False)

stop = timeit.default_timer()
print("Model trained in:",stop-start,"s")

weights

Model trained in: 13.6292988 s


array([-0.5788927 , -0.06935013,  0.2952298 ,  0.19996855,  0.12166342,
       -0.2609533 ,  0.33088139, -0.42333995,  0.17030403, -0.07630531,
        0.30812813])

Validating the model

In [15]:
predictions = modelBC.predict(testing, weights)
predictions

[[1,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  1],
 [1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,

Assessment of the model

In [16]:
modelBC.evaluate_acc(Preds=predictions)

Correct Predictions:  174
Total Predictions:  183
Accuracy Percentage:  95.082 %
Sensitivity Percentage:  83.3333 %


(95.08196721311475, 83.33333333333334)

Running kFolda Cross-Validation

In [17]:
start = timeit.default_timer() #Starting clock

modelBC.kfold(k=5,data=databc, LstFeatures=feats, OutCol=10, Rate=0.00001, MaxIter=1000, Reduct=1, Ran=False)

stop = timeit.default_timer()
print("kFold validation for Breast Cancer data ser performed in:",stop-start,"s")


Segmentation # 1  Results:
Correct Predictions:  109
Total Predictions:  136
Accuracy Percentage:  80.1471 %
Sensitivity Percentage:  62.2951 %

Segmentation # 2  Results:
Correct Predictions:  125
Total Predictions:  136
Accuracy Percentage:  91.9118 %
Sensitivity Percentage:  89.0625 %

Segmentation # 3  Results:
Correct Predictions:  126
Total Predictions:  136
Accuracy Percentage:  92.6471 %
Sensitivity Percentage:  85.7143 %

Segmentation # 4  Results:
Correct Predictions:  128
Total Predictions:  136
Accuracy Percentage:  94.1176 %
Sensitivity Percentage:  86.6667 %

Segmentation # 5  Results:
Correct Predictions:  136
Total Predictions:  139
Accuracy Percentage:  97.8417 %
Sensitivity Percentage:  91.4286 %

##### RESULTS FOR THE kFOLD CROSS VALIDATION WERE #####
Average Accuracy: 91.333 %
Average Sensitivity: 83.033 %
kFold validation for Breast Cancer data ser performed in: 93.3966482 s


## Red Wine Data Set

In the red wine quality data set, no instances with incorrect of missing values were found. Therefore, no example was removed from the data set

In [18]:
#Load data and remove missing values
filename = 'winequality_red.csv'
datawine = Read_File_DF(filename, separation=';', head=0, replace=["?"], drop=False)
datawine

Total number of missing/anomalous 'entries' in the data set:  0


array([[ 7.4  ,  0.7  ,  0.   , ...,  0.56 ,  9.4  ,  5.   ],
       [ 7.8  ,  0.88 ,  0.   , ...,  0.68 ,  9.8  ,  5.   ],
       [ 7.8  ,  0.76 ,  0.04 , ...,  0.65 ,  9.8  ,  5.   ],
       ...,
       [ 6.3  ,  0.51 ,  0.13 , ...,  0.75 , 11.   ,  6.   ],
       [ 5.9  ,  0.645,  0.12 , ...,  0.71 , 10.2  ,  5.   ],
       [ 6.   ,  0.31 ,  0.47 , ...,  0.66 , 11.   ,  6.   ]])

In this data set, the intercept value weights $ w_{0}$ were added as a new column to the beginning of the matrix. The resulting data matrix will have6 1599 rows and 12 columns: 10 features plus 1 intercept and the true labels

In [19]:
datawine = preproc(datawine, Index1=11, Index2=None, Flag=False)
datawine

array([[ 1.   ,  7.4  ,  0.7  , ...,  0.56 ,  9.4  ,  0.   ],
       [ 1.   ,  7.8  ,  0.88 , ...,  0.68 ,  9.8  ,  0.   ],
       [ 1.   ,  7.8  ,  0.76 , ...,  0.65 ,  9.8  ,  0.   ],
       ...,
       [ 1.   ,  6.3  ,  0.51 , ...,  0.75 , 11.   ,  1.   ],
       [ 1.   ,  5.9  ,  0.645, ...,  0.71 , 10.2  ,  0.   ],
       [ 1.   ,  6.   ,  0.31 , ...,  0.66 , 11.   ,  1.   ]])

Statistics on the data set

In [20]:
Data_Stats(datawine, QQ_DD = True, show = True)

STATS_FUNCTION_ERROR



Calling constructor for the 

In [21]:
modelWine = logisticReg(datawine)

Defining the set/subset of features to be selected (_always between 0-10 with the 0 included_ - $w_{0}$)

In [22]:
feats = [0,1,2,3,4,5,6,7,8,9,11]

Defining the set/subset of examples to be used in training and validating the model ( _for uses outside kFold method_ )

In [23]:
training = datawine[250:750,:]
testing = datawine[1000:1350,:]

Running model outside kFold method ( _single training and validation_ )

In [24]:
start = timeit.default_timer() #Starting clock to count time for training

weights = modelWine.fit_model(Training_Set=training, lstFeatures=feats, outCol=12, rate=0.00001, maxIter=100, reduct=1, ran=False)

stop = timeit.default_timer()
print("Model trained in:",stop-start,"s")

weights

Model trained in: 1.4517073000000096 s


array([ 0.00461939,  0.06990367, -0.00919313,  0.00779589,  0.02787056,
       -0.00026694,  0.07263767, -0.03266404,  0.00459079,  0.01499595,
        0.12718816])

Validating the model

In [25]:
predictions = modelWine.predict(testing, weights)
predictions

[[1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,


Assessment of the model

In [26]:
modelWine.evaluate_acc(Preds=predictions)

Correct Predictions:  229
Total Predictions:  350
Accuracy Percentage:  65.4286 %
Sensitivity Percentage:  93.6709 %


(65.42857142857143, 93.67088607594937)

Running kFolda Cross-Validation

In [28]:
start = timeit.default_timer() #Starting clock

modelWine.kfold(k=5,data=datawine, LstFeatures=feats, OutCol=12, Rate=0.0001, MaxIter=1000, Reduct=1, Ran=False)

stop = timeit.default_timer()
print("kFold validation for Wine Data set performed in:",stop-start,"s")

C:\Users\alloc\Anaconda3\envs\comp551a1\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
C:\Users\alloc\Anaconda3\envs\comp551a1\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log



Segmentation # 1  Results:
Correct Predictions:  194
Total Predictions:  319
Accuracy Percentage:  60.815 %
Sensitivity Percentage:  6.4516 %

Segmentation # 2  Results:
Correct Predictions:  125
Total Predictions:  319
Accuracy Percentage:  39.185 %
Sensitivity Percentage:  0.5128 %

Segmentation # 3  Results:
Correct Predictions:  165
Total Predictions:  319
Accuracy Percentage:  51.7241 %
Sensitivity Percentage:  98.0132 %

Segmentation # 4  Results:
Correct Predictions:  222
Total Predictions:  319
Accuracy Percentage:  69.5925 %
Sensitivity Percentage:  100.0 %

Segmentation # 5  Results:
Correct Predictions:  167
Total Predictions:  323
Accuracy Percentage:  51.7028 %
Sensitivity Percentage:  97.546 %

##### RESULTS FOR THE kFOLD CROSS VALIDATION WERE #####
Average Accuracy: 54.604 %
Average Sensitivity: 60.505 %
kFold validation for Wine Data set performed in: 195.76833439999996 s
